# Trabalho prático 2
### Luis Gustavo Lorgus Decker
### Luiz Antonio Falaguasta Barbosa

Importando as bibliotecas utilizadas

In [1]:
import numpy as np #Tratamento numérico
import matplotlib.pyplot as plt #Plot de gráficos
from sklearn import linear_model #Regressão linear
from sklearn import decomposition #Decomposição PCA
import PIL.Image as Image#Abrir imagens
import sklearn.metrics

Carregamos dados de treinamento
          
    Carregamos no vetor pixelsExpanded os tres canais de cada imagem.
    Cada elemento de 'pixelsExpanded' corresponde a uma imagem, e é uma matriz de 32x32 pixeis(tamanho da imagem)

In [2]:
#str(100).zfill(5)
#Ler arquivo de labels e parear com images.
labels = np.loadtxt(fname='cifar-10/train/labels',unpack=True)
#Ler as imagens
#As amostras estão nas linhas, as features estão nas colunas
datasetSize = 50000
nImages = 30000#Numero de imagens a serem carregadas(ATENÇÃO, MODIFICAR SEPARAÇÃO DE DADOS DE VALIDAÇÃO)
pixelsExpanded = np.empty((nImages,3073),int)#Cria a matriz de dados, com nImages linhas e 3073 colunas (uma para o label e 3072 para os pixeis rgb)
for img in range(0,nImages):#Para cada imagem 
    pixelsExpanded[img][0] = labels[img] #Atribui o label a primeira coluna
    imageIndex = str(img).zfill(5)
    filename = './cifar-10/train/' + imageIndex + '.png'#Cria o filename
    image = Image.open(fp=filename)#Carrega a imagem
    #print(image.size,image.getbands())
    pixels = image.getdata()#Acessa os pixeis
    #print(len(pixels))
    col = 1;#Indice atual do primeiro dado
    #Para cada imagem, ler os canais RGB e anexar a matrix de dados
    for pixel in range(0,len(pixels)):
        
        r,g,b = pixels[pixel][0:3] #Separa os canais
        #Adiciona o pixel
        pixelsExpanded[img][col] =r 
        pixelsExpanded[img][col+1] = g 
        pixelsExpanded[img][col+2] = b
        #print("Tratado imagem {} , pixel {} , ultimo indice {}".format(img,pixel,col+2))
        col = col+3 #Avança 3 posições
    #print("Processed {}% of the dataset".format(img/datasetSize))
print("Lidos os pixeis de {} imagens".format(nImages))

    

Lidos os pixeis de 30000 imagens


Separamos **40000** imagens como conjunto de treino, e **10000** como conjunto de validação.

In [3]:
trainingSetSize = 400 #Tamanho desejado do conjunto de treinamento
trainingSet = pixelsExpanded[0:trainingSetSize][:] #Separa os trainingSetSize primeiros dados como treinamento
validationSetSize = nImages - trainingSetSize #Tamanho do conjunto de Validação
validationSet = pixelsExpanded[trainingSetSize:][:] #Separa os validationSetSize dados após o ultimo usado em training set como validação.
del pixelsExpanded

# TODO
Aplicar o **PCA** nos dados

    A função PCA abaixo realiza uma redução linear de dimensionalidade utilizando uma decomposição de valores singulares (SVD). Assim, esta redução projeta os dados em um espaço de menor dimensionalidade.
    
    O SVD é realizado em cima da matriz de covariância dos dados, e retorna três matrizes:
    
    -A matriz U, que contém os autovetores dos quais escolhemos k que descrevem o hiperplano em Rk sob qual os dados serão projetados. Este k é escolhido de tal maneira que 'preservedVariance' seja a percentagem da variância dos dados preservada.
    
    -A matriz S, que contém os autovalores relacionados aos autovetores da matriz U. Estes autovetores representam a variância preservada em cada autovetor associado. Ou seja, escolhemos k autovetores de tal maneira que a soma de suas variancias sobre a soma da variancia total seja igual a 'preservedVariance'
    
    -A matriz V, que não utilizamos
    
    Utilizamos como método de calculo do SVD 'full', que realiza a decomposição completa. 
   

In [4]:
preservedVariance = 0.95
pca = decomposition.PCA(n_components=preservedVariance,svd_solver='full')
trainedPca = pca.fit(trainingSet[:,1:3072])
#ATENCAO
#Temos que guardar o pca treinado para executar a mesma
#redução de dimensionalidade no conjunto de validaçao/teste

Reduzimos a dimensionalidade do conjunto de treino

In [5]:
trainingSetReduced = trainedPca.transform(trainingSet[:,1:3072])
#print(trainingSetReduced.shape)
nFeatures = trainingSetReduced.shape[1]
print(nFeatures)


126


Reduzimos também a dimensionalidade do conjunto de validação

In [6]:
validationSetReduced = trainedPca.transform(validationSet[:,1:3072])

Para analisar o resultado e a eficiência do PCA, rodamos o algorítmo mantendo 99% da variância, 95% e 90%. 

**Para 99%**, reduzimos para 656 dimensões (2416 dimensões a menos).

**Para 95%**, reduzimos para 216 dimensões (2856 dimensões a menos).

**Para 90%**, reduzimos 99 para  dimensões (2973 dimensões a menos).

Escolhemos então seguir preservando 95% da variância, com uma redução de ~92% das dimensões.

Separamos também os dados das images de suas respectivas labels

In [7]:
trainlLabels = trainingSet[:,0]#Separa os labels do conjunto de treino
trainData = trainingSetReduced#Separa os dados do conjunto de treino
del trainingSetReduced
validationLabels = validationSet[:,0]#Separa os labels do conjunto de validação
validationData = validationSetReduced#Separa os dados do conjunto de validação




Inicializamos a regressão linear.

    Para classificação multiclasses, o algorítmo utiliza a metodologia "um vs resto" (parametro multi_class).

    O parâmetro C é o inverso da força de regularização. Valores menores implicam uma regularização mais forte. 

    Esta função também adiciona uma constante (bias) a função de decisão, permitindo um deslocamento na interceptação do eixo Y.

    Como método de regularização, utiliza L2-normalization. Neste método as penalizações aplicadas durante a atualização dos coeficientes é 
$$\begin{align}
\lambda * \sum_{i=1}^{k} ( \theta_i^2 )
\end{align}$$

    ou seja, a soma dos quadrados dos pesos. 
    
    
    Como função de minimização de custos, o classificador utiliza uma técnica de 'Coordinate descent', uma variante do gradiente descendente.
    
    max_iter especifica o número máximo de iterações que serão feitas visando minimizar a função de custo.


In [8]:
iteracoes = 1000
forca_regularizacao = 1
logreg=linear_model.LogisticRegression(C=forca_regularizacao,multi_class='ovr',max_iter=iteracoes, n_jobs=4)

In [9]:
print(trainData.shape)

(400, 126)


Treinamos o classificador com os dados de treinamento

In [10]:
logreg.fit(trainData,trainlLabels.astype('int'))

/usr/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 4.
  " = {}.".format(self.n_jobs))


LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=1000, multi_class='ovr', n_jobs=4,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [11]:
classified = logreg.predict(validationSetReduced)

In [12]:
sklearn.metrics.accuracy_score(validationLabels,classified)

0.22527027027027027